In [1]:
import os
import sys
import csv
import re
import math
import pandas as pd
# import pdb

import librosa
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib.style as ms
from tqdm import tqdm
import pickle

import IPython.display
import librosa.display
ms.use('seaborn-muted')
%matplotlib inline


In [ ]:
iemocap_full_release_path = "/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/IEMOCAP_full_release/"

iemocap_pre_processed_data_path = "/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/pre_processed_data/iemocap/"

In [ ]:
info_line = re.compile(r'\[.+\]\n', re.IGNORECASE)

start_times, end_times, wav_file_names, emotions, vals, acts, doms = [], [], [], [], [], [], []

for sess in range(1, 2):
    emo_evaluation_dir = iemocap_full_release_path + '/Session{}/dialog/EmoEvaluation/'.format(sess)
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        x = re.search("^Ses.*", file)
        if x == None:
            continue
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        info_lines = re.findall(info_line, content)
        for line in info_lines[1:]:  # the first line is a header
            #pdb.set_trace()   
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            start_time, end_time = start_end_time[1:-1].split('-')
            val, act, dom = val_act_dom[1:-1].split(',')
            val, act, dom = float(val), float(act), float(dom)
            start_time, end_time = float(start_time), float(end_time)
            start_times.append(start_time)
            end_times.append(end_time)
            wav_file_names.append(wav_file_name)
            emotions.append(emotion)
            vals.append(val)
            acts.append(act)
            doms.append(dom)

In [ ]:

df_iemocap = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'emotion', 'val', 'act', 'dom'])

df_iemocap['start_time'] = start_times
df_iemocap['end_time'] = end_times
df_iemocap['wav_file'] = wav_file_names
df_iemocap['emotion'] = emotions
df_iemocap['val'] = vals
df_iemocap['act'] = acts
df_iemocap['dom'] = doms

In [ ]:
df_iemocap

In [ ]:
sr = 44100
audio_vectors = {}

for sess in [1]:  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = '{}/Session{}/dialog/wav/'.format(iemocap_full_release_path, sess)
    #pdb.set_trace()
    orig_wav_files = os.listdir(wav_file_path)
    
#     print(orig_wav_files)
    for orig_wav_file in tqdm(orig_wav_files):
        #print("Wav file is ", orig_wav_file)
        x = re.search("^Ses.*", orig_wav_file)
        if '.DS_Store' == orig_wav_file:
            continue
        if x == None:
            print("Skiping file", orig_wav_file)
            continue
#         print(wav_file_path + orig_wav_file)
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)

            orig_wav_file, file_format = orig_wav_file.split('.')
            print("Working on file - ", orig_wav_file)
            #pdb.set_trace()
            for index, row in df_iemocap[df_iemocap['wav_file'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name, emotion, val, act, dom = row['start_time'], row['end_time'], row['wav_file'], row['emotion'], row['val'], row['act'], row['dom']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]


                X = librosa.stft(truncated_wav_vector)
                Xdb = librosa.amplitude_to_db(abs(X))
                plt.figure(figsize=(14, 5))
                #plt.figure()
                librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
                plt.colorbar()

                #pdb.set_trace()
                plt.savefig(iemocap_pre_processed_data_path + "images/" + str(emotion) +"_"+ str(val) +"_"+ str(act) +"_"+ str(dom) + '.png')
                plt.close('all')
                audio_vectors[truncated_wav_file_name] = truncated_wav_vector

        except:
            print('An exception occured for {}'.format(orig_wav_file))
    with open(iemocap_pre_processed_data_path + '/audio_vectors_exp_{}.pkl'.format(sess), 'wb') as f:
        pickle.dump(audio_vectors, f)

In [ ]:
with open("../small_data/pre_processed_data/audio_vectors_exp_1.pkl", 'rb') as file:
    d = pickle.load(file)

In [ ]:
len(d[list(d.keys())[-3]])

In [ ]:
df_iemocap

In [ ]:
len(d)

In [ ]:
!ls

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../small_data/pre_processed_data/iemocap/df_iemocap.csv")

In [ ]:
df.file_path[3]

In [ ]:
df.file_path[6]

In [ ]:
set(df.emotion)


In [ ]:
ems = ['ang', 'exc', 'fru', 'hap', 'neu', 'sur']

In [ ]:
ems

In [ ]:
'an' in ems

In [ ]:
ems = ['ang', 'hap', 'sad', 'fea', 'sur', 'neu']

In [2]:
import os
from glob import glob
import pandas as pd

In [11]:
enterfece_data_path = "/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/enterface_database"
pre_processed_data_path = '/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/pre_processed_data/enterface'
# pre_processed_data_path = "/home/student/keropyan/data/pre_processed_data/iemocap/"
# subject_num = 44
# subject_num = 2

emo_conv_dict = {
    'fear': 'fea',
    'surprise': 'sur',
    'sadness': 'sad',
    'happiness': 'hap',
    'anger': 'ang',
    'disgust': 'dis',
}


In [ ]:
def get_enterface_paths(enterfece_data_path):
    video_paths = []
    emotions = []
    for subject in os.listdir(enterfece_data_path):
        if subject == '.DS_Store':
            continue
    #     print(subject)
        sub_path = enterfece_data_path + '/' + subject
        for emotion in os.listdir(sub_path):
            if emotion == '.DS_Store':
                continue
            emo_path = sub_path + '/' + emotion

            for sent in os.listdir(emo_path):
                if sent == '.DS_Store':
                    continue

                sent_path = emo_path + '/' + sent
                paths = glob(sent_path + '/' + '*.avi')
                for p in paths:
                    video_paths.append(p)
                    emotions.append(emo_conv_dict[emotion])

    df_enterface = pd.DataFrame(columns=['file_path', 'emotion'])

    df_enterface['file_path'] = video_paths
    df_enterface['emotion'] = emotions

    df_enterface.to_csv(pre_processed_data_path + '/df_enterface.csv', index=False)


        

In [ ]:
get_enterface_paths(enterfece_data_path)

In [ ]:
df = pd.read_csv(pre_processed_data_path+'/df_enterface.csv')

In [ ]:
df.file_path[4]

In [6]:
ravdess_data_path = '/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/RAVDESS'
pre_processed_data_path = '/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/pre_processed_data/RAVDESS'



In [7]:
def get_ravdess_paths(ravdess_data_path, pre_processed_data_path):
    emotions = []
    actors = []
    video_paths = []
    for actor in os.listdir(ravdess_data_path):
        if actor == '.DS_Store':
            continue
        act_path = ravdess_data_path + '/' + actor


        ps = glob(act_path + '/' + '*.mp4')
        for p in ps:
            vals = p.split('/')[-1].split('.')[0].split('-')
            #filtering only videos with audio
            if vals[0] != '01':
                continue
            actors.append(vals[-1])
            video_paths.append(p)
            emotions.append(vals[2])

    df_ravdess = pd.DataFrame(columns=['file_path', 'emotion', 'actor'])

    df_ravdess['file_path'] = video_paths
    df_ravdess['emotion'] = emotions
    df_ravdess['actor'] = actors

    df_ravdess.to_csv(pre_processed_data_path + '/df_ravdess.csv', index=False)




In [8]:
get_ravdess_paths(ravdess_data_path, pre_processed_data_path)

In [9]:
df = pd.read_csv(pre_processed_data_path + '/df_ravdess.csv')

In [10]:
df

,file_path,emotion,actor
0,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,3,10
1,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,3,10
2,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,2,10
3,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,1,10
4,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,2,10
...,...,...,...
115,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,3,3
116,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,3,3
117,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,2,3
118,/Users/grigorkeropyan/Desktop/YSU_thesis/small...,2,3
